# Cropping arenas using OpenCV

in this notebook we demonstrate how arenas can be cropped out of an image using OpenCV tools.

# Packages import

In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil


In [12]:
print(len(os.listdir(f"experiment{experiment}")))


11


In [15]:
#deve esserci una cartella per ogni maze di ogni esperimento !!!!
#ciclo for va da zero a limite-1
#REMEMBER TO CALL THE FOLDER WITH IMAGES AS EXPERIMENT1, EXPERIMENT2 ETC (start from 1!!)


arenas = 9 #starts from zero so it's ok to have 9 here
experiments = 6 #remember experiment starts from 1 so if we have 5, this value has to be 6!!

for experiment in range(1,experiments): 
# create a folder for the experiment
    experiment_dir= f"mazes_experiment{experiment}" 
 # If the directory exists, delete it
    if os.path.exists(experiment_dir):
        shutil.rmtree(experiment_dir)
    #create directory
    os.makedirs(experiment_dir)

#images = len(os.listdir(f"experiment{experiment}"))  #starts from zero so add one more than the actual number of images 
    #for image in range(images):

    for image in range(len(os.listdir(f"experiment{experiment}"))):
    # Load the image
        img = cv2.imread(f"experiment{experiment}/image{image}.jpg") ##CHANGE PATH HERE 
       
# TO CHECK 
        '''fig, ax = plt.subplots()
        plt.axis("off")
        im = ax.imshow(img, cmap="gray", vmin=0, vmax = 255)
        plt.title(f"experiment{experiment}, image{image}")
        plt.show()'''

        # Convert to grayscale if not already
        if len(img.shape) > 2:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Enter the arenas coordinates
        X1 = 0
        X2 = 710
        X3 = 1450
        X4 = 2200
        X5 = 2980
        X6 = 3690

        Y1 = 0
        Y2 = 725
        Y3 = 1140
        Y4 = 1860
        Y5 = 2350
        Y6 = 3200

        # Make tuples containing the 9 combinations of coordinates required to get the arenas
        regions_of_interest = [
            (X1, Y1, X2, Y2),
            (X3, Y1, X4, Y2),
            (X5, Y1, X6, Y2),
            (X1, Y3, X2, Y4),
            (X3, Y3, X4, Y4),
            (X5, Y3, X6, Y4),
            (X1, Y5, X2, Y6),
            (X3, Y5, X4, Y6),
            (X5, Y5, X6, Y6),
        ]

        # Create a 3x3 grid of subplots to display each crop

        #fig, axs = plt.subplots(3, 3, figsize=(20, 20))
        
        '''for arena_number in range(arenas): #andranno da zero a 8
            
           axs[arena_number // 3, arena_number % 3].axis("off")
            axs[arena_number // 3, arena_number % 3].imshow(
                img[
                    regions_of_interest[arena_number][1] : regions_of_interest[arena_number][3],
                    regions_of_interest[arena_number][0] : regions_of_interest[arena_number][2],
                ],
                cmap="gray",
                vmin=0,
                vmax=255,
            )

        # Remove the axis of each subplot and draw them closer together
        for ax in axs.flat:
            ax.axis("off")
        plt.subplots_adjust(wspace=0, hspace=0)'''

        # Dictionary to store the images, useful for the plot
        arena_images = {}

        for arena_number in range(arenas): 
            # create a folder for the experiment
            arena_dir= f"{experiment_dir}/arena{arena_number}" 
            # If the directory exists, delete it
            #if os.path.exists(arena_dir):
                #shutil.rmtree(arena_dir)
            #create directory
            os.makedirs(arena_dir, exist_ok=True)

            #useful when you plot
            arena_name = f"experiment{experiment}_image{image}_Arena{arena_number}"

            # Crop the image
            arena= img[
                regions_of_interest[arena_number][1] : regions_of_interest[arena_number][3],
                regions_of_interest[arena_number][0] : regions_of_interest[arena_number][2],
            ]

        # Store the image in the dictionary, useful for the plot
            arena_images[arena_name] = arena

# TO CHECK 
            '''fig, ax = plt.subplots()
            plt.axis("off")
            im = ax.imshow(arena, cmap="gray", vmin=0, vmax = 255)
            plt.title(arena_name)
            plt.show()'''

            #LOOP TO DO IT WITH EACH single maze 
            # Dictionary to store the images
            maze_images = {}

            #threshold definition to understand what to crop 
            _, binary = cv2.threshold(arena,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
            #closing to remove black dots inside our kernel
            # Define the kernel for erosion
            kernel = np.ones((30, 30), np.uint8)  # Adjust kernel size as needed

            # Apply erosion
            closing = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

            # Apply connected components to label connected regions (so to extract the shape we are interested in) and get statistics
            num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(closing)

            # Iterate through each labeled component (excluding background label)
            for maze in range(1, num_labels):
                    
                #create subfolder of experiment, one for each maze
                maze_dir = f"{arena_dir}/maze{maze}"
                    # If the directory exists, delete it
                #if os.path.exists(maze_dir):
                    #shutil.rmtree(maze_dir)
                # Create the directory
                os.makedirs(maze_dir, exist_ok=True)

                # Get statistics for the component
                left, top, width, height, area = stats[maze]
                centroid_x, centroid_y = centroids[maze]

                # Create a mask for this specific component
                component_mask = np.uint8(labels == maze) * 255 
                
                # Use bitwise AND operation to extract the shape
                shape = cv2.bitwise_and(arena, arena, mask=component_mask)

                shape_crop= shape[top:top+height, left:left+width]
                maze_name = f"{arena_name}_maze{maze}"
                maze_images[maze_name] = shape_crop
                # Rotate the image
                if maze == 3:
                    shape_crop = cv2.rotate(shape_crop, cv2.ROTATE_180)

                # Save the image
                cv2.imwrite(f"{maze_dir}/{maze_name}.jpg", shape_crop)
#TO CHECK
                '''fig, ax = plt.subplots()
                plt.axis("off")
                im = ax.imshow(shape_crop, cmap="gray", vmin=0, vmax = 255)
                plt.title(maze_name)
                plt.show()'''

#extra code, jsut in case
        #     # Save or process the extracted shape
        #     cv2.imwrite(f'shape_{label}.jpg', shape)  
        #     # Print statistics
        #     print(f"Component {label}:")
        #     print(f"  Area: {area}")
        #     print(f"  Bounding Box: ({left}, {top}, {width}, {height})")
        #     print(f"  Centroid: ({centroid_x}, {centroid_y})")
        # # Show the result
        # cv2.imshow('Result', image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()         

FileNotFoundError: [WinError 3] Impossibile trovare il percorso specificato: 'experiment2'